In [1]:
import pandas as pd
import numpy as np

In [2]:
!pwd

/Users/lucasbraga/Documents/GitHub/active-learning/llm-prompt-based-validation


In [3]:
df_input = pd.read_csv("dummy_dataset.csv")

In [6]:
df_input

,business_name,address,city,business_category,has_website,address_present,confidence_score
0,Cafe Paris,320 Rue Saint-Augustin,Paris,Restaurant,True,True,0.82
1,Boulangerie du Pont,12 Rue de Rivoli,Paris,Bakery,False,True,0.74
2,Tech Hub,45 Avenue des Champs-Élysées,Paris,Electronics,True,False,0.65
3,Le Marché Vert,8 Rue Montorgueil,Paris,Grocery,False,False,0.22
4,Salon Élégance,5 Rue de la Paix,Paris,Salon,True,True,0.90
...,...,...,...,...,...,...,...
65,Pau Pyrenees,12 Rue Serviez,Pau,Other,False,False,0.22
66,La Rochelle Marine,14 Quai Duperré,La Rochelle,Sports,True,True,0.86
67,Amiens Cathédrale,3 Rue des Trois Cailloux,Amiens,Other,False,False,0.16
68,Brest Bretagne,6 Rue de Siam,Brest,Other,False,True,0.41


In [10]:
df_results = pd.read_json("results_openai.jsonl", lines=True)

# Expand the 'output' dict column into separate columns for easier analysis
output_df = pd.json_normalize(df_results['output'])
df_results = pd.concat([df_results.drop(columns=['output']), output_df], axis=1)

In [11]:
df_results

,input,raw_response,attempts,is_valid,confidence,reason
0,"{'business_name': 'Le Marché Vert', 'address':...","```json\n{\n ""is_valid"": false,\n ""confidenc...",1,False,22.0,Low confidence score and missing address verif...
1,"{'business_name': 'Boulangerie du Pont', 'addr...","```json\n{\n ""is_valid"": true,\n ""confidence...",1,True,74.0,"The venue is categorized as a Bakery, has a co..."
2,"{'business_name': 'Cafe Paris', 'address': '32...","```json\n{\n ""is_valid"": true,\n ""confidence...",1,True,82.0,"The venue is categorized as a Restaurant, has ..."
3,"{'business_name': 'Tech Hub', 'address': '45 A...","```json\n{\n ""is_valid"": false,\n ""confidenc...",1,False,65.0,"Address is not present, making the venue non-v..."
4,"{'business_name': 'Salon Élégance', 'address':...","```json\n{\n ""is_valid"": true,\n ""confidence...",1,True,90.0,"The venue is categorized as a salon, has a web..."
...,...,...,...,...,...,...
65,"{'business_name': 'Pau Pyrenees', 'address': '...","```json\n{\n ""is_valid"": false,\n ""confidenc...",1,False,22.0,"The venue lacks a specific business category, ..."
66,"{'business_name': 'La Rochelle Marine', 'addre...","```json\n{\n ""is_valid"": false,\n ""confidenc...",1,False,86.0,Miscategorized business category; 'Sports' doe...
67,"{'business_name': 'Brest Bretagne', 'address':...","```json\n{\n ""is_valid"": false,\n ""confidenc...",1,False,41.0,"The business category 'Other' is too vague, an..."
68,"{'business_name': 'Amiens Cathédrale', 'addres...","```json\n{\n ""is_valid"": false,\n ""confidenc...",1,False,16.0,"The venue lacks a specific business category, ..."


In [13]:
df_results['is_valid'].value_counts()

is_valid
False    39
True     31
Name: count, dtype: int64